In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [2]:
spam = pd.read_csv("spam.csv")
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
spam.shape

(5572, 2)

In [6]:
label_encoder = LabelEncoder()
categories = label_encoder.fit_transform(spam["Category"])

In [7]:
messages = spam["Message"].values
X_train, X_test, y_train, y_test = train_test_split(messages, categories, test_size=0.3)

In [8]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(X_train)
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [10]:
X_train = pad_sequences(X_train, padding="post", maxlen=500)
X_test = pad_sequences(X_test, padding="post", maxlen=500)

In [11]:
model = Sequential()

model.add(Embedding(input_dim=len(token.word_index),
                    output_dim=50,
                    input_length=500))
model.add(Flatten())
model.add(Dense(units=10, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation="sigmoid"))

In [12]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           368950    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 10)                250010    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 618971 (2.36 MB)
Trainable params: 618971 (2.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=10,
          verbose=True,
          validation_data=(X_test, y_test))

Epoch 1/500
390/390 [==============================] - 24s 42ms/step - loss: 0.0830 - accuracy: 0.8879 - val_loss: 0.0509 - val_accuracy: 0.9629
Epoch 2/500
390/390 [==============================] - 3s 9ms/step - loss: 0.0409 - accuracy: 0.9697 - val_loss: 0.0406 - val_accuracy: 0.9779
Epoch 3/500
390/390 [==============================] - 2s 6ms/step - loss: 0.0324 - accuracy: 0.9792 - val_loss: 0.0343 - val_accuracy: 0.9815
Epoch 4/500
390/390 [==============================] - 3s 8ms/step - loss: 0.0266 - accuracy: 0.9831 - val_loss: 0.0315 - val_accuracy: 0.9761
Epoch 5/500
390/390 [==============================] - 2s 6ms/step - loss: 0.0222 - accuracy: 0.9841 - val_loss: 0.0288 - val_accuracy: 0.9779
Epoch 6/500
390/390 [==============================] - 3s 7ms/step - loss: 0.0198 - accuracy: 0.9859 - val_loss: 0.0229 - val_accuracy: 0.9844
Epoch 7/500
390/390 [==============================] - 2s 5ms/step - loss: 0.0147 - accuracy: 0.9921 - val_loss: 0.0214 - val_accuracy: 0.98

In [15]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"loss = {loss} accuracy = {accuracy}")

53/53 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 0.9850
loss = 0.01421177014708519 accuracy = 0.9850478172302246


In [16]:
prediction = model.predict(X_test)
print(prediction)

53/53 [==============================] - 0s 2ms/step
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [17]:
predict = (prediction > 0.5)
print(predict)

[[False]
 [False]
 [False]
 ...
 [ True]
 [False]
 [False]]


In [19]:
cm = confusion_matrix(y_test, predict)
print(cm)

[[1408    7]
 [  18  239]]
